In [1]:
import numpy as np
import matplotlib.pyplot as plt
# import scipy.fftpack as spfft
import time

In [53]:
Xpts = int(1e3)+1
Xlngth = 10
X = np.linspace(0, Xlngth, Xpts)
dx = Xlngth/(Xpts-1)

Tpts = int(1e3)+1
Tlngth = 1
T = np.linspace(0, Tlngth, Tpts)[:-1]
dt = Tlngth/(Tpts-1)

lmbd = dt/dx
snaps = 10
snap = 0

mean_n = 3 / (4 * np.pi)
Gamma_0 = 1
kappa_0 = 1
therm_cond = .01

In [23]:
def l(array):
        return np.roll(array, 1, axis=-1)  # Has Been Checked

def r(array):
    return np.roll(array, -1, axis=-1)  # Has Been Checked

def ddx(array):  # Has Been Checked
    return (array - l(array)) / dx

def d2dx2(array):  # Has Been Checked
    return (r(array) - 2*array + l(array)) / (dx * dx)

In [69]:
n, ntot, nint, nflux, nfluxtot = np.zeros((2, Xpts)), np.zeros((snaps + 1, 2, Xpts)), np.zeros((snaps + 1, 2)), np.zeros((2, Xpts)), np.zeros((snaps + 1, 2, Xpts))
v, vtot, vint, vflux, vfluxtot = np.zeros((2, Xpts)), np.zeros((snaps + 1, 2, Xpts)), np.zeros((snaps + 1, 2)), np.zeros((2, Xpts)), np.zeros((snaps + 1, 2, Xpts))
e, etot, eint, eflux, efluxtot = np.zeros((2, Xpts)), np.zeros((snaps + 1, 2, Xpts)), np.zeros((snaps + 1, 2)), np.zeros((2, Xpts)), np.zeros((snaps + 1, 2, Xpts))
phi, phimtx, phitot = np.zeros((2, Xpts)), np.zeros((2, Xpts)), np.zeros((snaps + 1, 2, Xpts))

# Initial Conditions
ICchoice = np.array([1, 1, 1]) # Gaussian = 0, Wave = 1, Random = 2, Equilibrium = 3
ICdict = np.array([
        np.array([
            n_0 * np.ones(N) + .01 * np.exp(-(x - L / 2) ** 2) - .01 * np.sqrt(np.pi),
            np.ones(N) + .01 * np.exp(-(x - L / 2) ** 2) - .01 * np.sqrt(np.pi),
            np.ones(N) + .01 * np.exp(-(x - L / 2) ** 2) - .01 * np.sqrt(np.pi)
        ]),
        np.array([
            n_0 * np.ones(N) + .01 * np.exp(-(x - L / 2) ** 2) - .01 * np.sqrt(np.pi),
            np.ones(N) + .01 * np.cos(3*IC_freq * x),
            np.ones(N) + .01 * np.cos(3*IC_freq * x)
        ]),
        np.array([
            n_0 * np.ones(N) + .01 * np.random.random(N),
            np.ones(N) + .01 * np.random.random(N),
            np.ones(N) + .01 * np.random.random(N)
        ])
        np.array([
            n_0 * np.ones(N),
            np.zeros(N),
            np.zeros(N)
        ])
    ])

n[:] = ICdict[ICchoice[0]]
v[:] = ICdict[ICchoice[1]]
e[:] = ICdict[ICchoice[2]]

def savesnap():
    ntot[snap] = n
    vtot[snap] = v
    etot[snap] = e

# TODO: Check versus Modified Thomas Algorithm
phimtx = np.diag(2*np.ones(N))
phimtxrow, phimtxcol = np.indices(phimtx.shape)
phimtx[np.abs(phimtxrow - phimtxcol) == 1] = 1.
phimtx[-1,0] = 1
phimtx[0,-1] = 1

b = -4*np.pi*dx*dx*(n - mean_n)

phi = np.array([np.linalg.solve(phimtx, b[ii]) for ii in range(2)])


SyntaxError: invalid syntax (1809456235.py, line 24)

[[  0.17145888   0.37061972   0.51175044   0.60215194   0.72870025
    0.91209422   1.0695267    1.24099027   1.46068957   1.69506601
    1.92416895   2.19178351   2.42073843   2.58748125   2.7198705
    2.80690167   2.93674372   3.0831297    3.25721023   3.4704289
    3.64526926   3.78761996   3.88383859   3.98552952   4.1192212
    4.2883169    4.40878073   4.49319271   4.57825088   4.67059013
    4.72139038   4.83113406   4.94045858   5.0485692    5.10640682
    5.19788707   5.28268045   5.38258528   5.51348571   5.5987008
    5.66623059   5.76183026   5.86171974   5.93275998   6.01746738
    6.12612392   6.26265079   6.40407658   6.5871622    6.80100819
    7.01139656   7.25793465   7.56096754   7.90557108   8.26793373
    8.63706746   8.98190343   9.34311586   9.73284763  10.06769709
   10.38084574  10.74902926  11.06257511  11.35722843  11.66791422
   11.9924422   12.30868317  12.63014674  12.95986564  13.33025619
   13.73581943  14.19085296  14.5851403   14.92400731  15.29375975